In [4]:
import numpy as np

# npz 파일 불러오기
data = np.load("../data/processed/npz/fall_dataset.npz")

# 포함된 배열 이름(키) 확인
print(data.files)

['X', 'Y']


In [6]:
for key in data.files:
    print(f"{key}: shape={data[key].shape}, dtype={data[key].dtype}")

X: shape=(603, 80, 132), dtype=float32
Y: shape=(603,), dtype=int32


In [8]:
for key in data.files:
    value = data[key]
    print(f"\n===== 🔹 {key} =====")
    print(f"데이터 타입: {type(value)}")
    print(f"배열 길이: {len(value)}")

    # value가 넘파이 배열이면 shape 확인
    if isinstance(value, np.ndarray):
        if hasattr(value[0], 'shape'):
            print(f"각 샘플 shape: {value[0].shape}")
        else:
            print("단일 값 혹은 리스트 형태")

    # 실제 내용 일부 출력
    print("\n▶ 데이터 예시:")
    if isinstance(value, np.ndarray):
        print(value[:2])  # 앞의 2개만 출력 (너무 길면 잘림)
    else:
        print(value)


===== 🔹 X =====
데이터 타입: <class 'numpy.ndarray'>
배열 길이: 603
각 샘플 shape: (80, 132)

▶ 데이터 예시:
[[[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.27293772  0.6265868   0.08700023 ...  0.83788586 -0.00574968
    0.7727751 ]
  [ 0.27303737  0.6265299   0.09210216 ...  0.8377004  -0.01325505
    0.76665986]
  [ 0.2730993   0.62594956  0.09104376 ...  0.8372298   0.01354664
    0.75752795]]

 [[ 0.31872207  0.23558328 -0.42529473 ...  0.69121736  0.16975142
    0.94010687]
  [ 0.32702133  0.24053772 -0.40278172 ...  0.6913808   0.18703678
    0.93140626]
  [ 0.33841783  0.23897314 -0.39329156 ...  0.7007806   0.17574376
    0.92353743]
  ...
  [ 0.6550083   0.6550693  -0.0456286  ...  0.8437597  -0.4253331
    0.9848828 ]
  [ 0.6565796   0.6547311  -0.01307719 ...  0.84413743 -0.42496118
    0.98613954]
  [ 0

In [10]:
# 셀 1: imports + seed + 경로
import os, random
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from tqdm.notebook import tqdm

# 재현성
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# 데이터 경로 (환경에 맞게 수정)
BASE_DIR = Path.cwd().parent   # 예: src에서 실행하면 프로젝트 루트
NPZ_PATH = BASE_DIR / "data" / "processed" / "npz" / "fall_dataset.npz"

# 디바이스
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


In [12]:
# 셀 2: npz 로드 및 확인
data = np.load(NPZ_PATH)
print("keys:", data.files)   # 보통 ['X','Y']

X = data['X']   # (N, T, D)
Y = data['Y']   # (N,)

print("X dtype, shape:", X.dtype, X.shape)
print("Y dtype, shape:", Y.dtype, Y.shape)
print("positive (fall) count:", int(Y.sum()), "negative count:", len(Y)-int(Y.sum()))

keys: ['X', 'Y']
X dtype, shape: float32 (603, 80, 132)
Y dtype, shape: int32 (603,)
positive (fall) count: 146 negative count: 457


In [14]:
# 셀 3: split (샘플 단위 stratify)
test_ratio = 0.1
val_ratio = 0.1

# stratify로 클래스 비율 유지
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, Y, test_size=test_ratio, random_state=42, stratify=Y
)

val_relative = val_ratio / (1 - test_ratio)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=val_relative, random_state=42, stratify=y_trainval
)

print("train:", X_train.shape, y_train.shape)
print("val:  ", X_val.shape, y_val.shape)
print("test: ", X_test.shape, y_test.shape)

train: (481, 80, 132) (481,)
val:   (61, 80, 132) (61,)
test:  (61, 80, 132) (61,)


In [16]:
# 셀 4: scaler fit on train only -> transform all
Ntr, T, D = X_train.shape
scaler = StandardScaler()
scaler.fit(X_train.reshape(-1, D))  # (Ntr*T, D)

def apply_scaler(X_arr, scaler):
    N, T, D = X_arr.shape
    X2 = X_arr.reshape(-1, D)
    X2 = scaler.transform(X2)
    return X2.reshape(N, T, D)

X_train = apply_scaler(X_train, scaler)
X_val   = apply_scaler(X_val, scaler)
X_test  = apply_scaler(X_test, scaler)

# scaler 저장 (추론 시 필요)
import joblib
joblib.dump(scaler, BASE_DIR / "src" / "scaler.save")
print("Scaler saved.")

Scaler saved.


In [18]:
# 셀 5: Dataset & DataLoader
class FallDataset(Dataset):
    def __init__(self, X_np, y_np):
        self.X = X_np.astype(np.float32)  # (N,T,D)
        self.y = y_np.astype(np.int64)    # (N,)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]      # (T, D)
        y = self.y[idx]
        return torch.from_numpy(x), torch.tensor(y, dtype=torch.long)

batch_size = 32
train_loader = DataLoader(FallDataset(X_train, y_train), batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=False)
val_loader   = DataLoader(FallDataset(X_val, y_val), batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=False)
test_loader  = DataLoader(FallDataset(X_test, y_test), batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=False)

In [20]:
# 셀 6: LSTM 모델
class FallLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, dropout=0.3, num_classes=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True, dropout=dropout, bidirectional=False)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        # x: (B, T, D)
        out, (h_n, c_n) = self.lstm(x)  # out: (B, T, hidden_size)
        last = out[:, -1, :]            # 마지막 타임스텝 feature (B, hidden)
        logits = self.fc(last)          # (B, num_classes)
        return logits

input_size = X_train.shape[2]
model = FallLSTM(input_size=input_size).to(device)
print(model)

FallLSTM(
  (lstm): LSTM(132, 128, num_layers=2, batch_first=True, dropout=0.3)
  (fc): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=64, out_features=2, bias=True)
  )
)


In [22]:
# 셀 7: loss / optimizer / scheduler
# 클래스 불균형 보정
unique, counts = np.unique(y_train, return_counts=True)
print("train class counts:", dict(zip(unique, counts)))
n = len(y_train)
w0 = n / (2 * counts[0]) if counts[0] > 0 else 1.0
w1 = n / (2 * counts[1]) if len(counts)>1 and counts[1]>0 else 1.0
weights = torch.tensor([w0, w1], dtype=torch.float32).to(device)
print("class weights:", weights.cpu().numpy())

criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

train class counts: {0: 365, 1: 116}
class weights: [0.65890414 2.0732758 ]


In [24]:
# 셀 8: train/eval helpers
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    losses = []
    preds_all, labels_all = [], []
    for Xb, yb in tqdm(loader, desc="Train", leave=False):
        Xb = Xb.to(device); yb = yb.to(device)
        optimizer.zero_grad()
        logits = model(Xb)
        loss = criterion(logits, yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()

        losses.append(loss.item())
        preds = torch.argmax(logits.detach(), dim=1).cpu().numpy()
        preds_all.extend(preds); labels_all.extend(yb.cpu().numpy())

    avg_loss = float(np.mean(losses))
    acc = accuracy_score(labels_all, preds_all)
    prec, rec, f1, _ = precision_recall_fscore_support(labels_all, preds_all, average='binary', zero_division=0)
    return avg_loss, acc, prec, rec, f1

def eval_one_epoch(model, loader, criterion, device):
    model.eval()
    losses = []
    preds_all, labels_all = [], []
    with torch.no_grad():
        for Xb, yb in tqdm(loader, desc="Eval", leave=False):
            Xb = Xb.to(device); yb = yb.to(device)
            logits = model(Xb)
            loss = criterion(logits, yb)
            losses.append(loss.item())

            preds = torch.argmax(logits, dim=1).cpu().numpy()
            preds_all.extend(preds); labels_all.extend(yb.cpu().numpy())

    avg_loss = float(np.mean(losses))
    acc = accuracy_score(labels_all, preds_all)
    prec, rec, f1, _ = precision_recall_fscore_support(labels_all, preds_all, average='binary', zero_division=0)
    cm = confusion_matrix(labels_all, preds_all)
    return avg_loss, acc, prec, rec, f1, cm

In [26]:
# 셀 9: training loop
num_epochs = 30
best_val_f1 = 0.0
ckpt_dir = BASE_DIR / "models"
ckpt_dir.mkdir(parents=True, exist_ok=True)
best_ckpt = ckpt_dir / "best_fall_model.pth"

history = {"train_loss":[], "val_loss":[], "train_f1":[], "val_f1":[]}

for epoch in range(1, num_epochs+1):
    print(f"\n=== Epoch {epoch}/{num_epochs} ===")
    tr_loss, tr_acc, tr_prec, tr_rec, tr_f1 = train_one_epoch(model, train_loader, criterion, optimizer, device)
    print(f"Train: loss={tr_loss:.4f} acc={tr_acc:.3f} prec={tr_prec:.3f} rec={tr_rec:.3f} f1={tr_f1:.3f}")

    val_loss, val_acc, val_prec, val_rec, val_f1, cm = eval_one_epoch(model, val_loader, criterion, device)
    print(f"Val:   loss={val_loss:.4f} acc={val_acc:.3f} prec={val_prec:.3f} rec={val_rec:.3f} f1={val_f1:.3f}")
    print("Confusion matrix:\n", cm)

    history["train_loss"].append(tr_loss); history["val_loss"].append(val_loss)
    history["train_f1"].append(tr_f1); history["val_f1"].append(val_f1)

    scheduler.step(val_f1)

    # best 저장
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save({
            "epoch": epoch,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "val_f1": val_f1
        }, str(best_ckpt))
        print("Saved best checkpoint:", best_ckpt)

print("Training finished. Best val F1:", best_val_f1)
# history 저장
import joblib
joblib.dump(history, str(ckpt_dir / "history.joblib"))


=== Epoch 1/30 ===


Train:   0%|          | 0/16 [00:00<?, ?it/s]

Train: loss=0.5373 acc=0.807 prec=0.567 rec=0.836 f1=0.676


Eval:   0%|          | 0/2 [00:00<?, ?it/s]

Val:   loss=0.2834 acc=0.934 prec=0.789 rec=1.000 f1=0.882
Confusion matrix:
 [[42  4]
 [ 0 15]]
Saved best checkpoint: C:\Users\un747\capstone1_AI\models\best_fall_model.pth

=== Epoch 2/30 ===


...

Train:   0%|          | 0/16 [00:00<?, ?it/s]

Train: loss=0.0237 acc=0.992 prec=1.000 rec=0.966 f1=0.982


Eval:   0%|          | 0/2 [00:00<?, ?it/s]

Val:   loss=0.1145 acc=0.967 prec=1.000 rec=0.867 f1=0.929
Confusion matrix:
 [[46  0]
 [ 2 13]]
Training finished. Best val F1: 0.967741935483871


['C:\\Users\\un747\\capstone1_AI\\models\\history.joblib']

In [38]:
# 셀 10: load best and test
ck = torch.load(str(best_ckpt), map_location=device)
model.load_state_dict(ck["model_state"])
model.to(device)
model.eval()

test_loss, test_acc, test_prec, test_rec, test_f1, test_cm = eval_one_epoch(model, test_loader, criterion, device)
print("\n=== TEST RESULTS ===")
print(f"Loss: {test_loss:.4f} Acc: {test_acc:.3f} Prec: {test_prec:.3f} Rec: {test_rec:.3f} F1: {test_f1:.3f}")
print("Confusion matrix:\n", test_cm)

Eval:   0%|          | 0/2 [00:00<?, ?it/s]


=== TEST RESULTS ===
Loss: 0.0570 Acc: 0.967 Prec: 1.000 Rec: 0.867 F1: 0.929
Confusion matrix:
 [[46  0]
 [ 2 13]]


In [32]:
# 셀 11: inference helper (한 시퀀스 예측)
import joblib

scaler_path = BASE_DIR / "src" / "scaler.save"  # 앞서 저장한 scaler
scaler = joblib.load(scaler_path) if scaler_path.exists() else None

def preprocess_single_sequence(kps_seq, scaler=None):
    """
    kps_seq: numpy (T, D) raw keypoint flattened per-frame (same format as saved)
    returns: (1, T, D) float32
    """
    X = kps_seq.astype(np.float32)
    if scaler is not None:
        X = scaler.transform(X)  # scaler was fit on (N*T, D)
    return X[np.newaxis, ...]

def predict_sequence(model, seq_np):
    model.eval()
    x = torch.from_numpy(seq_np.astype(np.float32)).to(device)
    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        pred = int(np.argmax(probs))
    return pred, probs

# 사용 예:
# sample_seq = X_test[0]           # 이미 preprocessed/scaled
# pred, probs = predict_sequence(model, sample_seq[np.newaxis,...])
# print(pred, probs)